In [1]:
import csv
import time

import requests
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook
import pandas as pd





In [2]:
def get_val(tag, term):
    val = tag.find(term)['value'].encode('utf-8')
    return val

In [39]:
base = 'http://www.boardgamegeek.com/xmlapi2/thing?id={}&stats=1'
with open('ids.txt') as f:
    ids = [line.strip() for line in f.readlines()]
split = 10




In [4]:
los = []

In [60]:
ids_to_fetch = ids[:]

In [61]:
for i in tqdm_notebook(range(0, len(ids_to_fetch[:]), split)):

    for iteration in range(10):
        try:
            url = base.format(','.join(ids_to_fetch[i:i+split]))
            print('Requesting {}'.format(url))
            req = requests.get(url, timeout=30)
            soup = BeautifulSoup(req.content, 'xml')
            items = soup.find_all('item')
            for item in items:
                gid = item['id']
                gtype = item['type']
                gname = get_val(item, 'name')
                gyear = get_val(item, 'yearpublished')
                gmin = get_val(item, 'minplayers')
                gmax = get_val(item, 'maxplayers')
                gplay = get_val(item, 'playingtime')
                gminplay = get_val(item, 'minplaytime')
                gmaxplay = get_val(item, 'maxplaytime')
                gminage = get_val(item, 'minage')
                usersrated = get_val(item.statistics.ratings, 'usersrated')
                avg = get_val(item.statistics.ratings, 'average')
                bayesavg = get_val(item.statistics.ratings, 'bayesaverage')
                owners = get_val(item.statistics.ratings, 'owned')
                traders = get_val(item.statistics.ratings, 'trading')
                wanters = get_val(item.statistics.ratings, 'wanting')
                wishers = get_val(item.statistics.ratings, 'wishing')
                numcomments = get_val(item.statistics.ratings, 'numcomments')
                numweights = get_val(item.statistics.ratings, 'numweights')
                avgweight = get_val(item.statistics.ratings, 'averageweight')
                types = ",".join([x['name'] for x in item.statistics.ratings.findAll('rank')])
                categories = ",".join([x['value'] for x in item.findAll(type='boardgamecategory')])
                mechanics = ",".join([x['value'] for x in item.findAll(type='boardgamemechanic')])
                designers = ",".join([x['value'].encode('utf-8') for x in item.findAll(type='boardgamedesigner')])

                # desc = item.description.text.encode('ascii', 'ignore')
                los.append(pd.Series((gid, gtype, gname, gyear, gmin, gmax, gplay, gminplay, gmaxplay, gminage,
                                 usersrated, avg, bayesavg, owners, traders, wanters, wishers, numcomments,
                                 numweights, avgweight, types, categories, mechanics, designers)))
            time.sleep(.5)
            break
        except requests.exceptions.ReadTimeout:
            print('Timeout')
            time.sleep(20)
            continue
        except requests.exceptions.ConnectionError:
            print('ConnectionError')
            time.sleep(20)
            continue
    

Requesting http://www.boardgamegeek.com/xmlapi2/thing?id=60605,108623,60606,60601,71668,108626,86530,108264&stats=1



TypeError: 'NoneType' object has no attribute '__getitem__'

In [62]:
df = pd.DataFrame(los)
df.columns =              ['id', 'type', 'name', 'yearpublished', 'minplayers', 'maxplayers', 'playingtime',
                           'minplaytime', 'maxplaytime', 'minage', 'users_rated', 'average_rating',
                           'bayes_average_rating', 'total_owners', 'total_traders', 'total_wanters',
                           'total_wishers', 'total_comments', 'total_weights', 'average_weight','types',
                           'categories', 'mechanics', 'designers']
df

,id,type,name,yearpublished,minplayers,maxplayers,playingtime,minplaytime,maxplaytime,minage,...,total_traders,total_wanters,total_wishers,total_comments,total_weights,average_weight,types,categories,mechanics,designers
0,89371,boardgame,The Russo-Georgian War of 2008,2010,1,2,90,90,90,12,...,0,4,4,5,1,3,boardgame,Modern Warfare,Hex-and-Counter,Bruce Costello
1,89376,boardgame,Früher oder Später,2009,2,5,30,30,30,8,...,1,0,0,0,1,1,boardgame,Card Game,Hand Management,Bernhard Weber
2,89378,boardgame,Supervivencia,1990,2,6,80,80,80,10,...,0,0,0,1,1,2,boardgame,"Dice,Exploration","Card Drafting,Roll / Spin and Move,Trading",
3,5988,boardgame,Wings Over France,1992,1,1,60,60,60,10,...,7,25,68,29,9,3.2222,"boardgame,wargames","Aviation / Flight,Wargame,World War I","Action Point Allowance System,Campaign / Battl...",Terry Goodchild
4,40134,boardgame,Le Tour Cycliste,0,2,6,0,0,0,8,...,0,0,1,0,0,0,boardgame,"Racing,Sports",,(Uncredited)
5,5982,boardgame,Mothra vs. Godzilla,1982,2,2,90,90,90,10,...,0,15,17,2,2,1.5,"boardgame,wargames","Science Fiction,Wargame",Hex-and-Counter,(Uncredited)
6,5983,boardgame,Xanadu,1987,2,4,90,90,90,10,...,0,0,4,1,0,0,boardgame,Fantasy,Area Movement,(Uncredited)
7,5980,boardgame,Tokimeki Memorial 2 Game of Life ('Jinsei Game'),1998,2,6,60,60,60,0,...,0,0,0,1,0,0,boardgame,Video Game Theme,Roll / Spin and Move,(Uncredited)
8,5981,boardgame,Tower of Druaga,0,2,4,45,45,45,0,...,0,0,4,0,0,0,boardgame,"Fantasy,Video Game Theme",Roll / Spin and Move,(Uncredited)
9,5986,boardgame,Ra,1981,2,2,120,120,120,13,...,1,11,18,10,2,1.5,"boardgame,abstracts",Abstract Strategy,,Marco Donadoni


In [63]:
# Remove duplicates (groupby gameId->first?)
df = df.groupby('id').first()

In [65]:
len(df)

90397

In [64]:
df.to_csv('games.csv', encoding='utf-8')